In [35]:
import numpy as np
import evaluate
import pickle

from utils import read_jsonl, continuous_string, label_distribution

from transformers import AutoModelForTokenClassification, AutoTokenizer, DataCollatorForTokenClassification, TrainingArguments, Trainer, pipeline, DistilBertTokenizerFast
from huggingface_hub import notebook_login
from datasets import Dataset, DatasetDict

from sklearn.model_selection import train_test_split

In [2]:
# Function to read NER data
def read_universal_NER(file_path):
    with open(file_path, 'r', encoding = 'utf-8') as infile:
        # Split into lines
        lines = infile.readlines()

        # Define lists to store data 
        sentences = []
        labels = []
        current_sentence = []
        current_labels = []

        # Iterate over lines
        for line in lines:

            line = line.strip() # Remove whitespace
            if not line: # Skip empty lines
                continue

            # Check if line starts with sentence ID
            if line.startswith('# sent_id'):
                if current_sentence:
                    sentences.append(current_sentence)
                    labels.append(current_labels)
                current_sentence = []
                current_labels = []

            # Check for token lines
            elif not line.startswith("#"):
                parts = line.strip().split('\t')
                current_sentence.append(parts[1])
                current_labels.append(parts[2])

        if current_sentence:
            sentences.append(current_sentence)
            labels.append(current_labels)

    # Flatten lists
    #sentences = sum(sentences, [])
    #labels = sum(labels, [])

    return sentences, labels

In [31]:
# test_sents, test_labels = read_universal_NER('train_dev_test_sets/test.iob2')
# train_sents, train_labels = read_universal_NER('train_dev_test_sets/train.iob2')
# dev_sents, dev_labels = read_universal_NER('train_dev_test_sets/dev.iob2')

with open('train_dev_test_sets/train_raw.pkl', 'rb') as infile:
    train = pickle.load(infile)

with open('train_dev_test_sets/dev_raw.pkl', 'rb') as infile:
    dev = pickle.load(infile)

with open('train_dev_test_sets/test_raw.pkl', 'rb') as infile:
    test = pickle.load(infile)


train

[{'id': 1,
  'text': 'Paris Saint-Germain have confirmed the signing of French winger Ousmane Dembele from Barcelona. Dembele had been in talks over extending his Barcelona contract before PSG voiced a desire to trigger the    50m release clause in the Frenchman\'s contract, and although that clause expired last month, Dembele\'s desire to seal the move saw Barcelona agree to the same terms. Having entered the final year of his contract at Camp Nou, Barcelona preferred to cash in on Dembele when it became clear he would leave for free next summer if he remained at the club. For Dembele, it\'s a return to Ligue 1 for the first time since he departed Rennes in 2016, with the 26-year-old inking a five-year contract at Parc des Princes. feed "I\'m delighted to be joining Paris Saint-Germain and can\'t wait to play for my new club," Dembele told club media. "I hope I can continue to grow here and make all the club\'s fans proud." President Nasser Al-Khelaifi added: "We are delighted to welc

In [36]:
def format_gold_to_iob2_bert(gold_data):
    tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-cased') # Define model for tokenization
    output = []
    line_idx = 1

    for document in gold_data:
        text = document['text']
        
        encoded = tokenizer(text, return_offsets_mapping = True) # Tokenize and get offsets
        tokens = tokenizer.convert_ids_to_tokens(encoded.input_ids) # Get token texts
        offsets = encoded.offset_mapping

        labels = sorted(document['label'], key = lambda x: x[0])  # Sort labels by start index

        # Prepare the tags
        tags = ['O'] * len(tokens)

        # Apply IOB tagging
        for start, end, label in labels:
            started = False
            for i, (token_start, token_end) in enumerate(offsets):
                if token_start >= start and token_end <= end:
                    if token_start == start or not started:
                        tags[i] = f'B-{label}'
                        started = True
                    else:
                        tags[i] = f'I-{label}'

        # Generate the formatted output
        for token_idx, token_text in enumerate(tokens):
            output.append(f"{line_idx}\t{token_text}\t{tags[token_idx]}\t-\t-")
            line_idx += 1

    output.append('\n')
    return "\n".join(output)

In [38]:
print(format_gold_to_iob2_bert(test))

/Users/chris/myenv/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (1037 > 512). Running this sequence through the model will result in indexing errors


1	[CLS]	O	-	-
2	A	O	-	-
3	year	O	-	-
4	is	O	-	-
5	an	O	-	-
6	awful	O	-	-
7	long	O	-	-
8	time	O	-	-
9	in	O	-	-
10	football	O	-	-
11	,	O	-	-
12	and	O	-	-
13	that	O	-	-
14	has	O	-	-
15	certainly	O	-	-
16	proved	O	-	-
17	the	O	-	-
18	case	O	-	-
19	for	O	-	-
20	F	B-PER	-	-
21	##olar	I-PER	-	-
22	##in	I-PER	-	-
23	Ba	I-PER	-	-
24	##log	I-PER	-	-
25	##un	I-PER	-	-
26	.	O	-	-
27	This	O	-	-
28	time	O	-	-
29	last	O	-	-
30	year	O	-	-
31	,	O	-	-
32	few	O	-	-
33	would	O	-	-
34	have	O	-	-
35	known	O	-	-
36	too	O	-	-
37	much	O	-	-
38	about	O	-	-
39	the	O	-	-
40	young	O	-	-
41	Arsenal	B-ORG	-	-
42	striker	O	-	-
43	,	O	-	-
44	but	O	-	-
45	following	O	-	-
46	an	O	-	-
47	impressive	O	-	-
48	loan	O	-	-
49	spell	O	-	-
50	with	O	-	-
51	Stade	B-ORG	-	-
52	Re	I-ORG	-	-
53	##ims	I-ORG	-	-
54	in	O	-	-
55	Ligue	B-ORG	-	-
56	1	I-ORG	-	-
57	,	O	-	-
58	the	O	-	-
59	22	O	-	-
60	-	O	-	-
61	year	O	-	-
62	-	O	-	-
63	old	O	-	-
64	is	O	-	-
65	now	O	-	-
66	one	O	-	-
67	of	O	-	-
68	the	O	-	-
69	most	O	-	-
70	exciting	O	-	-

In [4]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/chris/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
# Helper function to process data
def preprocess_data(sents, labels):
    # Flatten the list of lists
    flat_data = [word for sublist in sents for word in sublist]
    flat_labels = [label for sublist in labels for label in sublist]

    # Join the words to form a single string
    text = ' '.join(flat_data)

    # Tokenize the text into sentences
    sentences = sent_tokenize(text)

    # Split the sentences into lists of words
    split_sentences = [word_tokenize(sentence) for sentence in sentences]

    # Align labels to the split sentences
    split_labels = []
    label_index = 0
    for sentence in split_sentences:
        sentence_labels = []
        for _ in sentence:
            if label_index < len(flat_labels):
                sentence_labels.append(flat_labels[label_index])
                label_index += 1
        split_labels.append(sentence_labels)
    
    return split_sentences, split_labels

In [6]:
processed_test_sents, processed_test_labels = preprocess_data(test_sents, test_labels)
processed_train_sents, processed_train_labels = preprocess_data(train_sents, train_labels)
processed_dev_sents, processed_dev_labels = preprocess_data(dev_sents, dev_labels)

In [212]:
# # Flatten the list of lists to get a single list of words
# flat_test_data = [word for sublist in test_sents for word in sublist]
# flat_test_labels = [label for sublist in test_labels for label in sublist]

# # Join the words to form a single string for sentence tokenization
# text = ' '.join(flat_test_data)

# # Tokenize the text into sentences
# sentences = sent_tokenize(text)

# # Split the sentences into lists of words
# test_sents_split = [sentence.split() for sentence in sentences]

In [213]:
# # Flatten the list of lists to get a single list of words
# flat_train_data = [word for sublist in train_sents for word in sublist]
# flat_train_labels = [label for sublist in train_labels for label in sublist]

# # Join the words to form a single string for sentence tokenization
# text = ' '.join(flat_train_data)

# # Tokenize the text into sentences
# sentences = sent_tokenize(text)

# # Split the sentences into lists of words
# train_sents_split = [sentence.split() for sentence in sentences]

In [214]:
# # Flatten the list of lists to get a single list of words
# flat_dev_data = [word for sublist in dev_sents for word in sublist]
# flat_dev_labels = [label for sublist in dev_labels for label in sublist]

# # Join the words to form a single string for sentence tokenization
# text = ' '.join(flat_dev_data)

# # Tokenize the text into sentences
# sentences = sent_tokenize(text)

# # Split the sentences into lists of words
# dev_sents_split = [sentence.split() for sentence in sentences]

In [215]:
# test_split_labels = []
# label_index = 0
# for sentence in test_sents_split:
#     sentence_labels = []
#     for _ in sentence:
#         if label_index < len(flat_test_labels):
#             sentence_labels.append(flat_test_labels[label_index])
#             label_index += 1
#     test_split_labels.append(sentence_labels)

In [216]:
# train_split_labels = []
# label_index = 0
# for sentence in train_sents_split:
#     sentence_labels = []
#     for _ in sentence:
#         if label_index < len(flat_train_labels):
#             sentence_labels.append(flat_train_labels[label_index])
#             label_index += 1
#     train_split_labels.append(sentence_labels)

In [217]:
# dev_split_labels = []
# label_index = 0
# for sentence in dev_sents_split:
#     sentence_labels = []
#     for _ in sentence:
#         if label_index < len(flat_dev_labels):
#             sentence_labels.append(flat_dev_labels[label_index])
#             label_index += 1
#     dev_split_labels.append(sentence_labels)

In [8]:
print(processed_test_sents[1])
print(processed_test_labels[1])

['This', 'time', 'last', 'year', ',', 'few', 'would', 'have', 'known', 'too', 'much', 'about', 'the', 'young', 'Arsenal', 'striker', ',', 'but', 'following', 'an', 'impressive', 'loan', 'spell', 'with', 'Stade', 'Reims', 'in', 'Ligue', '1', ',', 'the', '22', '-', 'year', '-', 'old', 'is', 'now', 'one', 'of', 'the', 'most', 'exciting', 'young', 'forwards', 'in', 'Europe', 'and', 'a', 'USMNT', 'international', '.']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'O', 'B-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'B-ORG', 'O']


In [198]:
# def split_list_and_labels(strings, labels, num_sublists):
#     # Initialize the sublists for strings and labels
#     sublists_strings = [[] for _ in range(num_sublists)]
#     sublists_labels = [[] for _ in range(num_sublists)]
    
#     # Distribute the strings and labels into sublists
#     for i, (string, label) in enumerate(zip(strings, labels)):
#         sublists_strings[i % num_sublists].append(string)
#         sublists_labels[i % num_sublists].append(label)
    
#     return sublists_strings, sublists_labels

In [199]:
# num_sublists = 800
# test_sents_split, test_labels_split = split_list_and_labels(test_sents[0], test_labels[0], num_sublists)
# train_sents_split, train_labels_split = split_list_and_labels(train_sents[0], train_labels[0], num_sublists)
# dev_sents_split, dev_labels_split = split_list_and_labels(dev_sents[0], dev_labels[0], num_sublists)

In [10]:
# Label Mappng
label2id = {
 'O': 0,
 'B-LOC': 1,
 'I-LOC': 2,
 'B-PER': 3,
 'I-PER': 4,
 'B-ORG': 5,
 'I-ORG': 6,
 'B-TIM': 7,
 'I-TIM': 8,
 'B-MON': 9,
 'I-MON': 10,
 'B-DAT': 11,
 'I-DAT': 12,
 'B-PCT': 13,
 'I-PCT': 14,
}

id2label = {v: k for k, v in label2id.items()}

In [11]:
label_names = [key for key in label2id.keys()]

In [12]:
def labels_map2id(labels, mapping):
    label_ids = []
    for sent in labels:
        label_id = [mapping.get(label, label) for label in sent]
        label_ids.append(label_id)
    return label_ids

In [13]:
train_label_ids = labels_map2id(processed_train_labels, label2id)
dev_label_ids = labels_map2id(processed_dev_labels, label2id)
test_label_ids = labels_map2id(processed_test_labels, label2id)

In [14]:
# Create datasets
train_dataset = Dataset.from_dict({"tokens": processed_train_sents, "ner_tags": train_label_ids})
dev_dataset = Dataset.from_dict({"tokens": processed_dev_sents, "ner_tags": dev_label_ids})
test_dataset = Dataset.from_dict({"tokens": processed_test_sents, "ner_tags": test_label_ids})

In [15]:
raw_datasets = DatasetDict({
    "train": train_dataset,
    "dev": dev_dataset,
    "test": test_dataset,
})

# Print to verify
print(raw_datasets)

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 1464
    })
    dev: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 408
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 451
    })
})


In [16]:
model_name = 'distilbert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

/Users/chris/myenv/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [234]:
#inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words = True, truncation = True)

In [233]:
#inputs.tokens()

In [236]:
#inputs.word_ids()

In [17]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [18]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs


### ÆNDRER ????

In [19]:
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=["tokens", "ner_tags"]
)

Map:   0%|          | 0/1464 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/451 [00:00<?, ? examples/s]

In [20]:
tokenized_datasets["test"]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 451
})

In [21]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
metric = evaluate.load("seqeval")

In [22]:
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [23]:
model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    id2label=id2label,
    label2id=label2id,
)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
notebook_login()

In [25]:
args = TrainingArguments(
    "bert-finetuned-ner-2",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

In [26]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["dev"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

  0%|          | 0/549 [00:00<?, ?it/s]

  0%|          | 0/51 [00:00<?, ?it/s]

/Users/chris/myenv/lib/python3.9/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/chris/myenv/lib/python3.9/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.8512877821922302, 'eval_precision': 0.0, 'eval_recall': 0.0, 'eval_f1': 0.0, 'eval_accuracy': 0.8005333333333333, 'eval_runtime': 3.7977, 'eval_samples_per_second': 107.434, 'eval_steps_per_second': 13.429, 'epoch': 1.0}


  0%|          | 0/51 [00:00<?, ?it/s]

...

{'eval_loss': 0.8457176089286804, 'eval_precision': 0.0, 'eval_recall': 0.0, 'eval_f1': 0.0, 'eval_accuracy': 0.8005333333333333, 'eval_runtime': 1.6609, 'eval_samples_per_second': 245.644, 'eval_steps_per_second': 30.705, 'epoch': 2.0}
{'loss': 0.8514, 'grad_norm': 1.3866020441055298, 'learning_rate': 1.7850637522768672e-06, 'epoch': 2.73}


  0%|          | 0/51 [00:00<?, ?it/s]

/Users/chris/myenv/lib/python3.9/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/chris/myenv/lib/python3.9/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.8423045873641968, 'eval_precision': 0.0, 'eval_recall': 0.0, 'eval_f1': 0.0, 'eval_accuracy': 0.8005333333333333, 'eval_runtime': 1.6922, 'eval_samples_per_second': 241.101, 'eval_steps_per_second': 30.138, 'epoch': 3.0}
{'train_runtime': 130.9111, 'train_samples_per_second': 33.549, 'train_steps_per_second': 4.194, 'train_loss': 0.8450003799410682, 'epoch': 3.0}


TrainOutput(global_step=549, training_loss=0.8450003799410682, metrics={'train_runtime': 130.9111, 'train_samples_per_second': 33.549, 'train_steps_per_second': 4.194, 'total_flos': 66936656286864.0, 'train_loss': 0.8450003799410682, 'epoch': 3.0})

In [246]:
trainer.push_to_hub(commit_message="Training complete 3")

CommitInfo(commit_url='https://huggingface.co/ChristianSneffeFleischer/bert-finetuned-ner-2/commit/23249e4cd671ba9a2df688f9fecaeb4931c1bc74', commit_message='Training complete 3', commit_description='', oid='23249e4cd671ba9a2df688f9fecaeb4931c1bc74', pr_url=None, pr_revision=None, pr_num=None)

In [247]:
eval_results = trainer.evaluate()
print(eval_results)

  0%|          | 0/56 [00:00<?, ?it/s]

{'eval_loss': 0.06387829780578613, 'eval_precision': 0.9284682080924855, 'eval_recall': 0.9420821114369502, 'eval_f1': 0.9352256186317323, 'eval_accuracy': 0.9851959806703996, 'eval_runtime': 1.8961, 'eval_samples_per_second': 235.226, 'eval_steps_per_second': 29.535, 'epoch': 3.0}


/Users/chris/myenv/lib/python3.9/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [248]:
# Make predictions on the test set
predictions, labels, metrics = trainer.predict(tokenized_datasets["test"])

  0%|          | 0/60 [00:00<?, ?it/s]

In [249]:
# Convert predictions to label indices
predictions = np.argmax(predictions, axis=-1)

In [250]:
# Ensure predictions is a list of lists
if isinstance(predictions, np.ndarray):
    if predictions.ndim == 1:
        predictions = predictions.tolist()
    else:
        predictions = [pred.tolist() for pred in predictions]

# Convert label indices to label names
true_predictions = [
    [id2label[p] for p in prediction]
    for prediction in predictions
]

In [255]:
# Function to write predictions
def write_iob2_predictions(test_sents, predictions, output_file):
    with open(output_file, 'w', encoding='utf-8') as outfile:
        for i, (sent, prediction) in enumerate(zip(test_sents, predictions)):
            outfile.write(f"# sent_id = test-{i+1:04d}\n")
            outfile.write(f"# text = {' '.join(sent)}\n")
            for j, (word, label) in enumerate(zip(sent, prediction)):
                outfile.write(f"{j+1}\t{word}\t{label}\t-\t-\n")
            outfile.write("\n")

In [254]:
# Write the shifted predictions to a file
write_iob2_predictions(processed_test_sents, true_predictions, 'baseline_preds/baseline_preds_football.iob2')

________________________________________________________________

In [3]:
# Testing

model_checkpoint = "ChristianSneffeFleischer/bert-finetuned-ner"
token_classifier = pipeline(
    "token-classification", model=model_checkpoint, aggregation_strategy="simple"
)
token_classifier("In Argentina , beef is revered , respected , and praised .")

/Users/chris/myenv/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[{'entity_group': 'LOC',
  'score': 0.99479514,
  'word': 'Argentina',
  'start': 3,
  'end': 12}]